In [3]:
%load_ext autoreload
%autoreload 2


In [226]:
import sys
!{sys.executable} -m pip install tables

     |████████████████████████████████| 4.3 MB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 135.0 MB/s eta 0:00:01


# DATA SEARCH

In [4]:
import pandas as pd
import datetime
import json 

## Extracting uni domains

In [1]:
df_unis = pd.read_csv('unis_v2.csv')
unique_domains = df_unis['domain'].unique()
domain_size = len(unique_domains)

d_time = datetime.datetime(2019, 12, 30, 12, 4, 5)
end_date = datetime.datetime(2020, 4, 1, 12, 4, 5).date()

search_domains = []
search_d = ''
for index in range(domain_size):
    if index != 0 and (index % 30 == 0 or index == domain_size - 1):
        search_domains.append(search_d[:-4])
        search_d = ''
    else:
        dom = unique_domains[index]
        if type(dom) == str:
            search_d += 'site:{} OR '.format(dom)

# data = {}
# date = d_time.date()

# while date < end_date:
# #     date1 = d_time.date()
#     d_time += datetime.timedelta(days=1)
#     date = d_time.date()
#     urls = []
#     print(date1, date)

#     for domains in search_domains:
#         query = 'Covid-19 ' + domains
#         results = search(query,None,  num=50, start=0, stop=100, pause=2, tbs=get_tbs(date, date))
#         for url in results:
#             urls.append(url)
#     data[date] = urls
#     with open('output_{}.json'.format(date), 'w') as f:
#     # this would place the entire output on one line
#     # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
#         json.dump(data, f)

In [13]:
search_domains

['site:abdn.ac.uk OR site:aber.ac.uk OR site:anglia.ac.uk OR site:aston.ac.uk OR site:bangor.ac.uk OR site:bathspa.ac.uk OR site:beds.ac.uk OR site:bbk.ac.uk OR site:bcu.ac.uk OR site:bolton.ac.uk OR site:bournemouth.ac.uk OR site:brad.ac.uk OR site:brighton.ac.uk OR site:bris.ac.uk OR site:brunel.ac.uk OR site:buckingham.ac.uk OR site:cam.ac.uk OR site:canterbury.ac.uk OR site:cardiff.ac.uk OR site:chester.ac.uk OR site:chi.ac.uk OR site:city.ac.uk OR site:coventry.ac.uk OR site:cranfield.ac.uk OR site:ucreative.ac.uk OR site:cumbria.ac.uk OR site:dmu.ac.uk OR site:derby.ac.uk OR site:dundee.ac.uk',
 'site:uea.ac.uk OR site:uwl.ac.uk OR site:edgehill.ac.uk OR site:ed.ac.uk OR site:essex.ac.uk OR site:exeter.ac.uk OR site:falmouth.ac.uk OR site:gla.ac.uk OR site:gcal.ac.uk OR site:glos.ac.uk OR site:glyndwr.ac.uk OR site:gold.ac.uk OR site:gre.ac.uk OR site:harper-adams.ac.uk OR site:hw.ac.uk OR site:herts.ac.uk OR site:hud.ac.uk OR site:hull.ac.uk OR site:ucl.ac.uk OR site:keele.ac.uk

## google search via 3rd party API's 

In [33]:
import requests
end_date = datetime.datetime(2020, 4, 1, 21, 4, 5).date()

# checkpoint
d_time = datetime.datetime(2020, 2, 10, 10, 4, 5)
date = d_time.date()


print(len(dates))
data = {}
for date in dates:
#     date1 = d_time.date()
#     d_time += datetime.timedelta(days=1)
#     date = d_time.date()
    urls = []
    print(date)
    date_txt = "{}-{}-{}".format(date.year, date.month, date.day)

    for index, domains in enumerate(search_domains):
        query = 'Covid-19 ' + domains
        params = {
            "query" : query,
            'access_key': '9e20b8648efe9aac5a8c461bcef60032',
            "engine":"google",
            "auto_location":'0',
            "period":'custom',
            "period_start": date.strftime('%Y-%m-%d'),
            "period_end": date.strftime('%Y-%m-%d'), 
            "num":"5000"
        }
        response = requests.get('http://api.serpstack.com/search', params=params);
        result = response.json()
#         result = serpwow.get_json(params)
        results_json = json.dumps(result, indent=2, sort_keys=True)
        ##
        with open('data_3/out_b{}_{}.json'.format(index, date), 'w') as f:
            f.write(results_json)
            
        ##
        urls.append(results_json)
    data[date] = urls

28
2020-02-17
2020-02-21
2020-02-24
2020-02-27
2020-02-28
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31


# DATA LOAD

In [25]:
import glob
end_date = datetime.datetime(2020, 4, 1, 21, 4, 5).date()


path = r'data_1/*.json'
dates = []
data = []
for file in glob.glob(path):
    tmp = json.loads(open(file).read())
    date = tmp['search_parameters']['time_period_max']
    try:
#         final_records = []
#         if len(tmp['organic_results']) == 50:
#             dt = datetime.datetime.fromisoformat(date + ' 10:10:10').date()
#             if dt < end_date:
# #                 print(date)
#                 dates.append(dt)
        records = tmp['organic_results']
        for x in records:
            info = {}
            info['search_date'] = date
            info['cached_page_link'] = x['cached_page_link'] if 'cached_page_link' in x.keys() else '' 
            info['title'] = x['title']
            info['snippet'] = x['snippet']
            info['link'] = x['link']
            info['domain'] = x['domain']
            data.append(info)
    except KeyError as e:
        print(e, 'file', file)
dates
df3 = pd.DataFrame(data)

'organic_results' file data_1\out_b3_2020-01-04.json


In [26]:
df = pd.concat([df1, df2, df3]) #df for data from data_1 folder

In [27]:
print(df.shape)
df.drop_duplicates(subset=['title', 'link', 'search_date'], inplace=True)
df.shape

(18139, 6)


(13548, 6)

In [29]:
df.to_csv('search_data_v3.csv', index='ignore')

## extracting content from webpages

In [8]:
import traceback
import logging
from tika import parser
import pandas as pd

In [5]:
df = pd.read_csv('search_data_v3.csv')

In [361]:
link = 'https://www.aclweb.org/anthology/K19-1079.pdf'
pdfFile = parser.from_file(link)
content = pdfFile["content"]


2020-05-01 15:05:03,637 [MainThread  ] [INFO ]  Retrieving https://www.aclweb.org/anthology/K19-1079.pdf to /tmp/anthology-k19-1079.pdf.
INFO:tika.tika:Retrieving https://www.aclweb.org/anthology/K19-1079.pdf to /tmp/anthology-k19-1079.pdf.


In [13]:
pdfFile['metadata']['title']

'Do Massively Pretrained Language Models Make Better Storytellers?'

In [28]:
import traceback
import logging
from tika import parser
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
df = pd.read_csv('search_data_v3.csv')
df['page_title'] = ''
df['page_text'] = ''
df['page_type'] = ''

starting_index = 1529
ending_index = 5000
for index, row in df.iloc[starting_index:ending_index].iterrows():
    
    if index >starting_index and index % 2000 == 0: 
        df.to_csv('search_results_v4_checkpoint_{}'.format(index))
    if index % 500 == 0:
        print('current index', index)
    try:
        df.at[index,'page_type'] = 'pdf'
        if link.endswith('.pdf'):
            row['page_type'] = 'pdf'
            pdfFile = parser.from_file(link)
            text = pdfFile["content"]
            tile = pdfFile['metadata']['title'] if 'title' in pdfFile['metadata'] else ''
        else:
            df.at[index,'page_type'] = 'html'
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "html")
            
            if soup.title:
                title = soup.title.string
            else: 
                title = ''
            # kill all script and style elements
            for script in soup(["script", "style"]):
                script.decompose()    # rip it out

            # get text
            text = soup.get_text()
            # break into lines and remove leading and trailing space on each
            lines = (line.strip() for line in text.splitlines())
            # break multi-headlines into a line each
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            # drop blank lines
            text = ' '.join(chunk for chunk in chunks if chunk)

        df.at[index,'page_title'] = title
        df.at[index,'page_text'] = text
    except requests.exceptions.SSLError as e:
        pass
    except Exception as e: 
        logging.error('index: ' + str(index) + ' ' +  traceback.format_exc())
df.to_csv('search_results_v4_{}_{}.csv'.format(starting_index, ending_index))

current index 2500
current index 3000
current index 3500
current index 4000
current index 4500


In [27]:
df.to_csv('search_results_v4_{}_{}.csv'.format(starting_index, index))

2180

In [160]:
x = df[df.page_text.duplicated(keep=False)]

x = x.dropna(subset=['page_text'])
x=x[x['Unnamed: 0.1'].astype(int)>5000]
x.shape

(139, 11)

In [88]:
sorted(x['Unnamed: 0.1'].astype(int))[-10:]

[8552, 8557, 8568, 8569, 8617, 8687, 8734, 8756, 8972, 9034]

In [97]:
x

,Unnamed: 0,Unnamed: 0.1,cached_page_link,domain,link,search_date,snippet,title,page_title,page_text,page_type
5,5.0,5,https://webcache.googleusercontent.com/search?...,www.dow.cam.ac.uk,https://www.dow.cam.ac.uk/outreach/university-...,2020-02-17,"Feb 17, 2020 - University Taster Day (Year 9)....",University Taster Day (Year 9) | Downing Colle...,Year 9 University Taster Webinar | Downing Col...,Year 9 University Taster Webinar | Downing Col...,html
9,9.0,9,https://webcache.googleusercontent.com/search?...,www.dow.cam.ac.uk,https://www.dow.cam.ac.uk/outreach/year-9-univ...,2020-02-17,"Feb 17, 2020 - Year 9 University Taster Webina...",Year 9 University Taster Webinar | Downing Col...,Year 9 University Taster Webinar | Downing Col...,Year 9 University Taster Webinar | Downing Col...,html
12,12.0,12,https://webcache.googleusercontent.com/search?...,www.cranfield.ac.uk,https://www.cranfield.ac.uk/research/phd/intel...,2020-02-17,"Feb 17, 2020 - artificial intelligence, machin...",Intelligent Dynamic Flood Response and Recover...,403 - Forbidden: Access is denied.,403 - Forbidden: Access is denied. Server Erro...,html
13,13.0,13,https://webcache.googleusercontent.com/search?...,blog.derby.ac.uk,https://blog.derby.ac.uk/2020/02/being-a-unive...,2020-02-17,"Feb 17, 2020 - Ever wondered what it's like to...",What it's like to be a university residential ...,403 Forbidden,403 Forbidden 403 Forbidden nginx,html
15,15.0,15,http://webcache.googleusercontent.com/search?q...,0-blogs.biomedcentral.com.brum.beds.ac.uk,http://0-blogs.biomedcentral.com.brum.beds.ac....,2020-02-17,"Feb 17, 2020 - Nineteen studies were quantitat...",Highlights of the BMC Series: January 2020 - B...,403 Forbidden,403 Forbidden 403 Forbidden nginx,html
...,...,...,...,...,...,...,...,...,...,...,...
13471,13471.0,1522,https://webcache.googleusercontent.com/search?...,www.rncm.ac.uk,https://www.rncm.ac.uk/performance/an-evening-...,1/15/2020,"Jan 16, 2020 - This event has been cancelled a...",An Evening with Hilary Mantel - CANCELLED - Ro...,An Evening with Hilary Mantel - CANCELLED - Ro...,An Evening with Hilary Mantel - CANCELLED - Ro...,html
13485,13485.0,1537,https://webcache.googleusercontent.com/search?...,www.qub.ac.uk,https://www.qub.ac.uk/courses/undergraduate/20...,1/15/2020,"Jan 15, 2020 - ... this degree. As a result of...",Midwifery Sciences (BSC HONS) B720 | Courses ....,Midwifery Sciences (BSC HONS) B720 | Courses |...,Midwifery Sciences (BSC HONS) B720 | Courses |...,html
13486,13486.0,1538,https://webcache.googleusercontent.com/search?...,www.qub.ac.uk,https://www.qub.ac.uk/courses/undergraduate/20...,1/15/2020,"Jan 15, 2020 - ... this degree. As a result of...",Mental Health Nursing (BSC HONS) B760 | Course...,Mental Health Nursing (BSC HONS) B760 | Course...,Mental Health Nursing (BSC HONS) B760 | Course...,html
13487,13487.0,1539,https://webcache.googleusercontent.com/search?...,www.qub.ac.uk,https://www.qub.ac.uk/courses/undergraduate/20...,1/15/2020,"Jan 15, 2020 - ... this degree. As a result of...",Midwifery Sciences (BSC HONS) B720 | Courses |...,Midwifery Sciences (BSC HONS) B720 | Courses |...,Midwifery Sciences (BSC HONS) B720 | Courses |...,html


# Data Processing

## Processing extracted content - merge all contents into one df 

In [155]:
import glob

df = pd.DataFrame()
path = 'search_results_v4_checkpoints/*.csv'
for file in glob.glob(path):
#     print(file)
    name = file.replace('search_results_v4_checkpoints/search_results_v4_checkpoint_', '')
    name = name.replace('.csv', '')
    indices = name.split('_')
    ## get indices containing page info (title and body)
    starting_index = int(indices[0])
    ending_index = int(indices[1])
    
    ## load df 
    df_tmp = pd.read_csv(file)
    df_tmp = df_tmp.iloc[starting_index:ending_index]
    
    df = pd.concat([df_tmp, df])

In [156]:
df.shape

(13548, 11)

In [157]:
for index, item in df.iterrows():
    if not pd.isnull(df['page_text'].iloc[index]):
        if link.endswith('.pdf'):
            df.at[index,'page_type'] =  'pdf'
        else:
            df.at[index,'page_type'] =  'html'

In [158]:
# save to json
df.to_json('search_results_v4.json')

## Filter on Covid-19 Keywords

In [280]:
## keep instance that contain covid-related words
keywords_lst = ['COVID-19, coronavirus, self-isolation, infection, pandemic'
         ,'Covid–19, pandemic, Coronavirus, Government advice, crisis',
         'lockdown, self-isolate, cancelled'
       ]
keywords = []
for words in keywords_lst:
    keywords += words.lower().split(', ')
keywords = set(keywords)

df['contains_keyword'] = ''

for index, row in df.iterrows():
    body = str(row['page_text']).lower()
    title = str(row['page_title']).lower()
    if any(word in body+ ' ' + title for word in keywords):
        df.at[index,'contains_keyword'] = 1
    else:
        df.at[index,'contains_keyword'] = 0

In [281]:
df.contains_keyword.value_counts()

1    12021
0     1527
Name: contains_keyword, dtype: int64

In [285]:
## remove rows not matching keywords -> contains_keywords is 0
df = df[df['contains_keyword']==1]
df.shape

(12021, 12)

In [286]:
## save rows only containing covid keywrods to v5
df.to_csv('search_results_v5.csv')

In [340]:
# pd.to_datetime(df['search_date'], format='%Y-%m-%d')
## convert string to date format
dates = []
for index, row in df.iterrows(): 
    
    x = row['search_date']
    try:
        d = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    except ValueError as e:
        d = datetime.datetime.strptime(x, '%m/%d/%Y').date()
    
    df.at[index,'search_date'] = d


In [347]:
## retain records dated prior to april 1

date = datetime.datetime(2020, 4, 1, 21, 4, 5).date()
df = df[df['search_date'] <date]
df.shape


(11140, 12)

In [341]:
df.to_csv('search_results_v6.csv')

datetime.date(2020, 2, 17)

## Split text_page to sents and create an index for senteces -> doc_id (doci_id is index from original df)

In [348]:
import nltk
from spacy.lang.en import English
nlp = English(disable=['ner', 'textcat'])
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)
doc = nlp(x)
list(doc.sents)
df_sents = pd.DataFrame()
sents_idx = []
for index, row in df.iterrows():
    
    text = row['page_text']
    if type(text) == float: ## nan
        sents_idx += [('', index)]
    else:
        try:
            doc = nlp(text)
            sents = list(doc.sents)
            # map each list of sents to doc index
            sents = [(sent.string, index) for sent in sents]
        except ValueError as e:
            sents = nltk.sent_tokenize(text)
            sents = [(sent, index) for sent in sents]
            print('valueError, using nltk', index)
            
        sents_idx += sents

valueError, using nltk 6037


In [351]:

df_sents = pd.DataFrame(sents_idx, columns=['sent', 'doc_id'])
df_sents.to_hdf('sents_idx.hdf', key='sents_idx')
df_sents.shape

(416517, 2)

## data processing with USE

In [354]:
import pandas as pd

df_sents = pd.read_hdf('sents_idx.hdf', key='sents_idx')

## break into 2 indices
breakpoint = int(df_sents.shape[0]/2)
df_sents_1 = df_sents.iloc[:breakpoint]
df_sents_2 = df_sents.iloc[breakpoint:]

In [356]:
df_sents_1.iloc[0]

sent      Women in STEM Webinair | Downing College Cambr...
doc_id                                                    0
Name: 0, dtype: object

In [358]:

import tensorflow as tf

import tensorflow_hub as hub
import numpy as np

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [106]:
# dot product query vs embedded list
def sims(data_emb, input_text):
    embd = embed([input_text])[0].numpy()
    return np.dot(data_emb, embd)

AttributeError: '_UserObject' object has no attribute 'device'

In [161]:
# embed titles 
titles = df.page_title.to_list()
titles = [title if type(title)==str else '' for title in titles]
p = [0 if t=='' else 1 for t in titles]
p=np.array(p)
print(np.count_nonzero(p))

titles_emb = embed(titles)

13084


In [ ]:
## sents 

In [359]:
## embed body - index 1
sents_emb_1 = embed(df_sents_1.sent)

In [360]:
sents_emb_2 = embed(df_sents_2.sent)

In [163]:
query = 'guidelines and protective measures'; threshold = 0.2
results = sims(titles_emb, query)

# Get the indexes/indices of elements greater than 4 
idx = np.where( results> threshold)[0]
# idx = array([4, 5, 6, 7, 8])

# Get the elements of the array that are greater than 4
values = results[ results> threshold]
# elts = array([5, 6, 7, 8, 9])


for i, v in zip(idx, values):
    print(titles[i], v)


	Arts and Health Journal
 0.20829864

	Welcome and Orientation Activity
 0.23332433
Virtual Panel: Coronavirus and Conservation | Conservation Research Institute 0.21787846
Environmental Health and Preventive Medicine | Home 0.30716363
University of Essex - Enhanced Protection Measures «  Vice-Chancellor 0.29392666
Care and Support | CJD 0.2477134
Mitigation and Extenuating Circumstances 0.25607383
Digital Signage Content Guidelines 0.30545795
Facilities and Instruments 0.21730116
Aims and Assessment 0.23061979
Student Protection Plan 0.20849241
Policy and Impact 0.30500403
Refund policy 0.20789672
Change: Critical Understandings, Practices and Action 0.2572924
Mindfulness and counselling short courses 0.21104819
Policy and Guidance on Research Degrees | Academic Support 0.30499053
Learning and Assessment 0.23480977
Data Protection and Privacy Group (Disestablished) 0.2552428
Health and Safety Services Homepage 0.26678556
Atrial Fibrillation Management and Stroke Prevention 0.21355356

In [131]:
values = results[ results> 0.8]
values

array([0.99999976], dtype=float32)